In [2]:
import random
import numpy as np
import pandas as pd
from ckiptagger import WS
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer

# 讀取檔案
dataDF = pd.read_csv('./lyricDF(tokens).csv', encoding='utf8', index_col=0)

In [3]:
def getQueryTokens(query):
    '''query斷詞，並加進corpus中，回傳斷詞的series'''

    path = "C:/Users/Admin/miniconda3"  # 請自行更改為斷詞data所在的資料位置
    ws = WS(f"{path}/data")

    tokens = ws(query)

    # 把query文本加入coupus中
    tokenLIST=[]
    for token in tokens:
        tokenLIST.append(str(token))
    tokenSeries=pd.Series(tokenLIST)
    corpus=dataDF.Tokens.append(tokenSeries,ignore_index=True)

    return corpus

In [4]:
def recommand(corpus):
    vectorizer = TfidfVectorizer()
    tfidf_doc = vectorizer.fit_transform(raw_documents=corpus).toarray() #結果轉成陣列
    tfidfDF=pd.DataFrame(tfidf_doc)
    #len(vectorizer.vocabulary_)  #13025

    # TF-IDF向量置中
    tfidfDF = tfidfDF-tfidfDF.mean()

    # 維度縮減
    pca = PCA(n_components=20)
    pcaFit = pca.fit(tfidfDF)         # 減去平均值那個
    pca_topic_vectors = pcaFit.transform(tfidfDF)

    columns = [f"topic{i}" for i in range(pca.n_components)]

    pca_topic_vectorsDF = pd.DataFrame(pca_topic_vectors, columns=columns)


    # 計算各topic的標準差
    corpusSTD=pca_topic_vectorsDF.std()

    # 只考慮那些PCA>0的主題
    topic_pos=[]
    for i in range(20):
        if pca_topic_vectorsDF.loc[len(pca_topic_vectorsDF)-1][i] > 0 :
            topic_pos.append((pca_topic_vectorsDF.loc[len(pca_topic_vectorsDF)-1][i],i))


    # 取出PCA最大的5個topic
    topic_fit=sorted(topic_pos,reverse=True)[:5]


    # 選擇在2個標準差以內歌曲

    std=[]
    topicLIST=[]
    for pca, i in topic_fit:
        std.append(pca + corpusSTD[i]*2)
        topicLIST.append(f'topic{i}')
        
    newDF= pca_topic_vectorsDF[(pca_topic_vectorsDF[topicLIST[0]] > 0 ) &
                                (pca_topic_vectorsDF[topicLIST[0]] <= std[0]) &
                                (pca_topic_vectorsDF[topicLIST[1]] > 0 ) &
                                (pca_topic_vectorsDF[topicLIST[1]] <= std[1]) &
                                (pca_topic_vectorsDF[topicLIST[2]] > 0 ) &
                                (pca_topic_vectorsDF[topicLIST[2]] <= std[2]) &
                                (pca_topic_vectorsDF[topicLIST[3]] > 0 ) &
                                (pca_topic_vectorsDF[topicLIST[3]] <= std[3]) &
                                (pca_topic_vectorsDF[topicLIST[4]] > 0 ) &
                                (pca_topic_vectorsDF[topicLIST[4]] <= std[4]) ]

    # 符合推薦的歌曲index
    indexLIST=(list(newDF.index))


    # 隨機選擇其中5首歌
    pick=random.choices(indexLIST, k=5)

    #印出推薦歌曲
    print(f'推薦以下這五首歌：')
    print()
    for i in range(len(pick)) :
        print(f'第{i+1}首',end="  ")
        print(dataDF.Singer[pick[i]],end="  ")  
        print(dataDF.Title[pick[i]])
        print(dataDF.Lyric[pick[i]])
        print()
    print()

In [5]:
#請在此輸入想要推薦的內容。
query=['請推薦我快樂的歌。'] 

corpus=getQueryTokens(query)
recommand(corpus)

c:\Users\Admin\miniconda3\envs\pythonnlp2\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


推薦以下這五首歌：

第1首  周華健  有故事的人
走著　忍著　醒著　想著　看愛情悄悄近了 冷的　暖的　甜的　苦的　在心裡纏繞成河 曲折的心情有人懂　怎麼能不感動 幾乎忘了昨日的種種　開始又敢作夢 我決定不躲了　你決定不怕了　我們決定了讓愛像綠草原滋長著 天地遼闊相遇多難得　都是有故事的人才聽懂心裡的歌 我決定不躲了　你決定不怕了　就算下一秒坎坷這一秒是快樂的 曾經交心就非常值得　我要專注愛你不想別的沒有忐忑

第2首  林宥嘉  天上的男人
時代一旦瘋狂到面目猙獰 生不逢時也只能身不由己 人被命運連根拔起 再隨手散成飄萍或浮雲 靜好歲月過成了顛沛流離 好女好男哪還有選擇餘地 一個個生命被扭曲 變成了酸楚的傳奇 在天上的男人 借來了翅膀飛 憑著熱血就視死如歸 但羽翼驀然被天收回那一瞬間 回望來處 該說愛或虧欠 地上的女人們 遲早的未亡人 早把生活活成了生存 若偶爾想起沒跟上人生那段青春 淡然一笑 不是不恨 靜好歲月原來是上天悲憫 人有多殘忍人都難以置信 活生生千萬個悲劇 換幾行史冊的註記 在天上的男人 借來了翅膀飛 可惜不像候鳥去有回 當羽翼驀然被天收回那一瞬間 最後一眼 看見的美不美 地上的女人們 遲早的未亡人 一隻隻還活著的鬼魂 早擦乾淚痕藏好傷痕但求餘生 現世安穩 無戰止爭

第3首  華晨宇  環遊
我在 擁擠路口 聽人群爭吵不休 吵著無謂藉口 吵著向左向右 我環遊 這顆星球 看時間擺弄著沙漏 我不原地停留  我大步行走 走出精彩 雙腳飛 沒有疲憊 飛翔著倔強的花蕾 陽光飛 沒有傷悲 世界明媚 彼岸愛上尋找 而天空愛上了奔跑 我們怎敢睡著 我們怎敢牢騷 生命這無休旅程 然後的然後是追逐 看看誰會認輸 看誰會停步 我會一 路 奉 陪 *勇敢飛 擁吻雨水 逆風讓翱翔更珍貴 燦爛飛 擁吻沉醉 堅強花蕾 飛翔的花朵 天空的焰火 沒有太多言說 驕傲的沉默 行進若不退縮 夢不會隕落 飛翔的 自我 Repeat  *

第4首  周華健  上上籤
瞭解你　難如登天　我真不在乎再要多少時間 心若倒懸　仍感謝天　我最美的發現 男人婆　只是表面　其實你心思細過錦緞纏綿 日子久了　就明白　眾人中我還是首選 我得承認　男人有時蠢話連篇　多虧有你處處留了顏面 我有幾次心不在焉　將真話說得膚淺 男人有時蠢話連篇　越是在意越是想不周延 男人總是蠢話連篇　留在身邊討厭　沒有又掛念 感情事

In [5]:
#請在此輸入想要推薦的內容。
query=['心情愉悅開開心心，春暖花開高興愉快。'] 

corpus=getQueryTokens(query)
recommand(corpus)

c:\Users\Admin\miniconda3\envs\pythonnlp2\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


推薦以下這五首歌：

第1首  羅志祥  NO
No Cap! Real Talk Only 我言語不打太極 案例多的不勝枚舉 Totally thought I'm crazy Totally caught your eyes Nothing can stop me Cuz I'm too hot Too lit 突襲 前進 招集 著力 必須走起立竿見影 虛無 泡影 絕不成立 不樹敵 沈住氣 成助力 Only roll with the real Hold up! Bruh~ I'm for real, No joke No joke I'm for real, No joke No joke I'm the real deal, now you better listen up I Know, I Know You want to dissent dissent dissent dissent I Know, I Know You want to dissent dissent dissent dissent I rule the game! Boi Not just the fame! Boi 都蒙了 Boi 還傻傻相信啊 Boi Just turn it turn it Turn down 無知的懦弱 Clap clap 信真的不道聽途說 到處收集消息不是為了 證明 Who's fake 不在意不搖頭 並不表示 這是真的可以 Aye Aye! 不戳破 不是默許 Hold up!!! I'm for real, No joke No joke I'm for real, No joke No joke I'm the real deal, now you better listen up I Know, I Know You want to dissent dissent dissent dissent I Know, I Know You want to dissent dissent dissent dissent Hold up! Bruh~ I'm for real, No joke No joke I'm for real, No joke No joke I'm the real deal, now you better listen up I K